# Packages and paths

In [ ]:
# Packages and modules
import os
import sys
import numpy as np
import calc_poll as cp
import pandas as pd
import plotly.express as px

# Mount Google Drive, if running in Colab
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
    sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks/Canvas')

# Change this to the path for the current version of the course
if os.path.isdir('/Users/mmchenry/Documents/Teaching'):
    root = '/Users/mmchenry/Documents/Teaching/E109/2022_SS2'

# Here is an alternate path, on Google Drive
elif os.path.isdir('/content/drive/MyDrive/Teaching'):
    root = '/content/drive/MyDrive/Teaching/E109 Human Physiology/e109 S2022'

# And here is another possibility (e.g., for the TA)
elif os.path.isdir('/content/drive/MyDrive/Grad/teaching/e109s22/e109 S2022'):
    root = '/content/drive/MyDrive/Grad/teaching/e109s22/e109 S2022'

else:
    raise ValueError('root path not found. Need to add new root path.')


# Import scores

In [ ]:
# Path to canvas CSV file that includes updated roster
cn_path = root + os.path.sep + "canvas.csv"

# Check path
if not os.path.isfile(cn_path):
    raise ValueError('canvas.csv file not found')

# Open and read canvas data
cn_file = open(cn_path)
c = pd.read_csv(cn_file,skiprows=[1, 2])

# From the canvas file, capture student IDs
# cnID = c.xs("SIS Login ID",axis=1)

# From the canvas file, final score, trim header
cnScore = c.xs("Unposted Final Score",axis=1)

# Plot raw score distribution
# fig = px.histogram(cnScore, labels={"value"="Final score"})
fig = px.histogram(cnScore, x="Unposted Final Score", nbins = 100)
fig.layout.update(showlegend=False,bargap=0.05)
fig.update_xaxes(range=[10, 100], tick0=10, dtick=10, showgrid=True)
fig.show()

# Calculate cutoffs
Under the course settings in Canvas, input the grade cut-offs by clicking on 'Enable course grading scheme' and then inputting the values by clicking on 'view grading scheme'.

In [ ]:
# cuts = [max(nTotals); floor([mu+0.8*sigma mu-0.2*sigma mu-1.4*sigma mu-2.2*sigma])'];

# Extract final score values
s    = c.xs("Unposted Final Score",axis=1)
mu   = np.mean(s)
sig  = np.std(s)

# Standard cut-offs
bigCuts = np.array([np.max(s), mu+0.8*sig, mu-0.2*sig, mu-1.4*sig, mu-2.2*sig])

# COVID cut-off correction
# bigCuts[1:] = bigCuts[1:]-10

# Adjustment to cutoffs, when the mean is relatively high
print('Note: I am giving the students 5 bonus points in this grade calculation')
bigCuts[1:] = bigCuts[1:]-5

# Initialize cut-off values
cutVal    = np.array([])
# cutVal[0] = 100

for i in range(1,len(bigCuts)):
    rVal     = bigCuts[i-1] -bigCuts[i]
    cutVal   = np.append(cutVal, round(bigCuts[i] + 0.75*rVal,2))
    cutVal   = np.append(cutVal, round(bigCuts[i] + 0.25*rVal ,2))
    cutVal   = np.append(cutVal, round(bigCuts[i],2))

# Remove first Add floor value
cutVal = cutVal[1:]
cutVal = np.append(cutVal,0)

# Define data for the table
d = {'Grades': ['A','A-','B+','B','B-','C+','C','C-','D+','D','D-','F'],
     'Cutoff': cutVal}

# Create table of cutoffs
cuts = pd.DataFrame(d)

bigCuts[0] = 100

# Output the table
print(cuts)

# Plot cutoffs on distribution
fig = px.histogram(cnScore, x="Unposted Final Score", nbins = 100)
fig.layout.update(showlegend=False,bargap=0.05)
fig.update_xaxes(range=[10, 100], tick0=10, dtick=10, showgrid=True)
for cLine in cutVal:
    fig.add_vline(x=cLine,line_color='gray', line_dash="dash")
for cLine in bigCuts:
    fig.add_vline(x=cLine,line_color='white')
fig.show()

# Display roster scores
This offers the opportunity to check that the final grades in Canvas match expectation from the cutoff values.

In [ ]:
ltr_grades = pd.Series(dtype='str')

for c_score in s:

    if c_score>=cutVal[0]:
        idx = 0
    elif c_score<cutVal[-2]:
        idx = len(cutVal)-1
    else:
        idx = np.where((c_score>=cutVal[1:]) & (c_score<cutVal[:-1]))[0]+1

    # Check output
    if len([idx])==0:
        raise ValueError('No matching grade found')
    elif len([idx])>1:
        raise ValueError('More than one matching grade found')

    ltr_grades = pd.concat([ltr_grades, pd.Series(cuts.Grades[idx])])

# Reindex
new_idx           = np.arange(len(ltr_grades))
ltr_grades.index  = new_idx

# Create new dataframe
d_frame = {'Student': c.Student, 'Final_score': s, 'Final_grade': ltr_grades}
df_out = pd.DataFrame(d_frame)

df_out

# Check roster grades
After setting the cut-offs, export the grades again to check that they match my calculation.
One way that discrepancies can arise is if "Automatically apply grade for missing submissions" is not set under "Gradebook settings". Note that the grade for missing assignments should be 0%
See information below about the different types of final scores.

In [ ]:
# Set 'no problem variable'
no_prob = True

# Loop thru my listing of students
for i_student in df_out.index:

    # Extract all final grades for i_student
    c_student          = c.xs("Student",axis=1)[i_student]
    c_unposted_final    = c.xs("Unposted Final Grade",axis=1)[i_student]
    c_unposted_current = c.xs("Unposted Current Grade",axis=1)[i_student]
    c_final_grade       = c.xs("Final Grade",axis=1)[i_student]
    my_student         = df_out.Student[i_student]
    my_grade           = df_out.Final_grade[i_student]

    # Report mismatches
    if c_student is not my_student:
        raise ValueError('Student names are not matching between my roster and Canvas')

    if (c_unposted_final != c_unposted_current):
        print(c_student + ": Unposted Final Grade is not the same as Unposted Current Grade in Canvas")
        no_prob = False

    if (c_unposted_final != c_final_grade):
        print(c_student + ": Unposted Final Score is not the same as Final Grade in Canvas")
        no_prob = False

    if (c_unposted_current != c_final_grade):
        print(c_student + ": Unposted Current Score is not the same as Final Grade in Canvas")
        no_prob = False

    if (my_grade != c_unposted_final):
        print(c_student + ": My final grade is not the same as Unposted Final Grade")
        no_prob = False

if no_prob:
    print('No discrepancies between the various final grades in Canvas and from my own calculations! Final grades ready to be posted.')
else:
    print(' ')
    print('Look into resolving the above discrepancies and then rerun this notebook with updated csv grade book from Canvas')


# Info on Canvas' confusing grade categories


This is from [this website](https://tuftsedtech.screenstepslive.com/s/18992/m/73355/l/1028758-understanding-canvas-gradebook-totals-and-exported-totals).

1. Current Score  Percentage score for graded, NOT MUTED, assignments (Note: John has a current score of 90% because he has only one graded, unmuted assignment, in which he received 45 out of 50 points (90%)

1. Unposted Current Score  Percentage score for all graded assignments (muted and unmuted) (Note: John has an unposted current score of 80% because he has only 2 graded assignments receiving 80 points out of a possible 100 points (80%)

1.  Final Score  Percentage score for all graded and ungraded assignments EXCLUDING muted “Grades”. “0” points are awarded for all un-submitted (ungraded) and muted assignments. (Note: John has a final score of 30% because he was awarded “45” points for paper 1, “0” points for paper 2 (muted) and “0” points for paper 3 - Total 45 out of 150 or 30%)

1. Unposted Final Score - Percentage score for all graded and ungraded assignments INCLUDING muted assignment grades, “0” points are awarded for all un-submitted (ungraded) assignments. (Note: John has an unposted final score of 53.33% because he was awarded “45” points for paper 1, “35” points for paper 2 and “0” points for paper 3  total 80 points out of 150 possible points (53.33%)

1. Current Grade  Letter Grade for graded, NOT MUTED, assignments (Note: John has a current grade of A- because he has only one graded, unmuted assignment, in which he received 45 out of 50 points (90% or A-)

1. Unposted Current Grade  Letter Grade for all graded assignment (muted and unmuted) (Note: John has an unposted current grade of B- because he has only 2 graded assignments receiving 80 points out of a possible 100 points (80% or B-)

1. Final Grade  Letter Grade for all graded and ungraded assignments EXCLUDING muted “Grades”. “0” points are awarded for all un-submitted (ungraded) and muted assignments. (Note: John has a final grade of F because he was awarded “45” points for paper 1, “0” points for paper 2 (muted) and “0” points for paper 3 - Total 45 out of 150 or 30% or F)

1. Unposted Final Grade  Letter Grade for all graded and ungraded assignments INCLUDING muted assignments, “0” points are awarded for all un-submitted (ungraded) assignment grades. (Note: John has an unposted final score of F because he was awarded “45” points for paper 1, “35” points for paper 2 and “0” points for paper 3  total 80 points out of 150 possible points (53.33% or F)